In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 

In [16]:
def extract_demographics(df, n, a):
    # Initialize dictionary to hold demographic data with placeholders for easier mapping
    demographic_data = {
        'ID': n,
        'Expert' : a,
        'Date': None,
        'Height_in': None,
        'Height_cm': None,
        'Weight_lbs': None,
        'Weight_kg': None,
        'Age': None,
        'Sex': None
    }
    if a == 2:
        return demographic_data

    empty = False

    for col in df.columns[-15:]:
        for index in range(20):
            cell_value = str(df.at[index, col]).strip()
            if 'DATE:' in cell_value:
                    col_index = df.columns.get_loc(col)
                    
                    # Extract and convert Date to datetime
                    date_str = df.iat[index, col_index + 1]
                    demographic_data['Date'] = pd.to_datetime(date_str, errors='coerce')
                    
                    height_in_str = df.iat[index + 1, col_index + 1]
                    weight_lbs_str = df.iat[index + 3, col_index + 1]


                    if (len(height_in_str) != 5):
                        empty = True
                        print(f"Failed to extract demographic data from Subgroup_I_{n}_{a}.xlsx -- Height = {height_in_str} - Weight = {weight_lbs_str}")
                    
                    if not empty:
                        try:
                            demographic_data['Height_in'] = float(height_in_str.split()[0]) if height_in_str else None
                            
                            # Extract Height in cm, remove 'cm', and convert to float
                            height_cm_str = df.iat[index + 2, col_index + 1]
                            demographic_data['Height_cm'] = float(height_cm_str.split()[0]) if height_cm_str else None
                                                    
                            # Extract Weight in lbs, remove 'lbs', and convert to float
                            weight_lbs_str = df.iat[index + 3, col_index + 1]
                            demographic_data['Weight_lbs'] = float(weight_lbs_str.split()[0]) if weight_lbs_str else None
                        
                            # Extract Weight in kg, remove 'kg', and convert to float
                            weight_kg_str = df.iat[index + 4, col_index + 1]
                            demographic_data['Weight_kg'] = float(weight_kg_str.split()[0]) if (weight_kg_str and len(weight_kg_str) > 2) else None
                            
                            # Extract Age and convert to integer
                            age_str = df.iat[index + 5, col_index + 1]
                            demographic_data['Age'] = int(age_str)
                            
                            # Extract Sex
                            demographic_data['Sex'] = df.iat[index + 6, col_index + 1].strip()
                        
                        except Exception as e:
                            print(f"Failed to extract demographic data from Subgroup_I_{n}_{a}.xlsx - {e}")

                    
                    break  # Break the loop once all demographic data is extracted
        if demographic_data['Date'] is not None:
            break



    return demographic_data


# Initialize a list to store all records
all_records = []

for n in range(1, 101):  
    for a in range(1, 3):  
        ##################### ADJUST PATH ###############################
        file_name = f"Data/Subgroup_I_{n}_{a}.xlsx"
        try:
            df = pd.read_excel(file_name)
            
            # Convert 'SpO2' and 'HR' columns to numeric, coercing errors to NaN
            df['SpO2'] = pd.to_numeric(df['SpO2'], errors='coerce')
            df['HR'] = pd.to_numeric(df['HR'], errors='coerce')

            # Convert "Events" to a categorical type
            df['Events'] = df['Events'].astype('category')
            
            # Extract demographic information
            demographics = extract_demographics(df, n, a)  # Assuming this function is defined as needed
            
            # Filter the DataFrame based on 'SpO2' and 'HR'
            filtered_df = df[(df['SpO2'] > 0) & (df['HR'] > 0)]
            
            # Calculate total epochs, average SpO2, and average HR
            tot_Epoch = filtered_df['Epoch'].count()
            avg_SpO2 = filtered_df['SpO2'].mean()
            avg_HR = filtered_df['HR'].mean()
            min_SpO2 = filtered_df['SpO2'].min()
            min_HR = filtered_df['HR'].min()
            max_SpO2 = filtered_df['SpO2'].max()
            max_HR = filtered_df['HR'].max()
            
            # Summarize 'BPOS', 'Stage', and now 'Events' as well
            bpos_counts = filtered_df['BPOS'].value_counts().to_dict()
            stage_counts = filtered_df['Stage'].value_counts().to_dict()
            events_summary = filtered_df['Events'].value_counts().to_dict()  
            
            # Combine all extracted data into a record
            record = {
                **demographics,
                
                'Total_Epoch': tot_Epoch,
                'Min_SpO2': min_SpO2,
                'Min_HR': min_HR,
                'Max_SpO2': max_SpO2,
                'Max_HR': max_HR,
                'Average_SpO2': avg_SpO2,
                'Average_HR': avg_HR,
                'BPOS_Summary': bpos_counts,
                'Stage_Summary': stage_counts,
                'Events_Summary': events_summary  
            }
            all_records.append(record)
        except Exception as e:
            print(f"Failed to process {file_name}: {e}")

# Convert the list of dictionaries to a DataFrame
final_df = pd.DataFrame(all_records)
print(final_df.head())

# Specify the desired output CSV file path
# output_csv_path = 'summary_data_subgroup_3.csv'
# final_df.to_csv(output_csv_path, index=False)

# print(f"Data successfully written to {output_csv_path}")

Failed to extract demographic data from Subgroup_I_7_1.xlsx -- Height = 0 in - Weight = nan
Failed to extract demographic data from Subgroup_I_10_1.xlsx -- Height = in - Weight = lbs
Failed to extract demographic data from Subgroup_I_17_1.xlsx -- Height = 180 in - Weight = 90 lbs
Failed to extract demographic data from Subgroup_I_21_1.xlsx -- Height = in - Weight = lbs
Failed to extract demographic data from Subgroup_I_23_1.xlsx -- Height = 152 in - Weight = 116 lbs
Failed to extract demographic data from Subgroup_I_27_1.xlsx -- Height = in - Weight = lbs
Failed to extract demographic data from Subgroup_I_32_1.xlsx -- Height = in - Weight = lbs
Failed to extract demographic data from Subgroup_I_33_1.xlsx -- Height = 1 in - Weight = 311 lbs
Failed to extract demographic data from Subgroup_I_41_1.xlsx - 'float' object has no attribute 'strip'
Failed to extract demographic data from Subgroup_I_48_1.xlsx -- Height = in - Weight = lbs
Failed to extract demographic data from Subgroup_I_49_1.

In [31]:
import pandas as pd
df = pd.read_csv("summary_data_subgroup_3.csv")
df

,ID,Expert,Date,Height_in,Height_cm,Weight_lbs,Weight_kg,Age,Sex,Total_Epoch,Min_SpO2,Min_HR,Max_SpO2,Max_HR,Average_SpO2,Average_HR,BPOS_Summary,Stage_Summary,Events_Summary
0,1,1,2009-03-28,NaN,NaN,NaN,NaN,30.0,Male,566,93,53.0,100,186.0,97.901060,69.489399,"{'B': 298, 'L': 268}","{'N2': 240, 'R': 109, 'N3': 106, 'N1': 56, 'W'...","{'AR': 52, 'Awake': 3, 'MH': 2, 'AR,AR': 1, 'A..."
1,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,570,93,53.0,100,186.0,97.905263,69.742105,"{'B': 297, 'L': 273}","{'N2': 257, 'N3': 103, 'R': 103, 'N1': 56, 'W'...","{'AR': 62, 'Awake': 5, '79 Awake': 0, 'AR,AR':..."
2,2,1,2009-04-25,69.0,175.0,198.0,90.0,41.0,Male,875,92,58.0,98,143.0,94.718857,70.235429,"{'R': 645, 'L': 195, 'B': 19, 'P': 11, 'N': 5}","{'N2': 305, 'N3': 197, 'R': 149, 'N1': 122, 'W...","{'AR': 70, 'PLM': 24, 'Awake': 8, 'SN': 6, 'MH..."
3,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,867,92,58.0,98,143.0,94.733564,70.229527,"{'R': 643, 'L': 189, 'B': 19, 'P': 11, 'N': 5}","{'N2': 294, 'N3': 242, 'R': 163, 'N1': 93, 'W'...","{'AR': 121, 'Awake': 17, 'AR,AR': 3, 'MChg': 1..."
4,3,1,2009-02-14,69.0,176.0,187.0,85.0,52.0,Male,820,91,50.0,98,123.0,94.524390,59.700000,"{'R': 406, 'L': 386, 'B': 28}","{'N3': 317, 'N2': 248, 'R': 103, 'W': 88, 'N1'...","{'AR': 55, 'OH': 13, 'MA': 9, 'MA,AR': 7, 'LM'..."
5,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,818,91,50.0,98,123.0,94.523227,59.679707,"{'R': 404, 'L': 386, 'B': 28}","{'N3': 288, 'N2': 260, 'R': 118, 'W': 80, 'N1'...","{'AR': 97, 'Awake': 25, 'AR,AR': 2, 'MChg': 2,..."
6,4,1,2009-03-14,NaN,NaN,NaN,NaN,58.0,Male,774,14,55.0,100,129.0,97.040052,70.124031,"{'L': 336, 'B': 158, 'R': 132, 'N': 8}","{'N2': 232, 'W': 162, 'N3': 159, 'N1': 133, 'R...","{'AR': 93, 'MH': 15, 'Awake': 6, 'AR,MH': 4, '..."
7,4,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,778,14,55.0,100,129.0,97.041131,70.131105,"{'L': 355, 'B': 164, 'R': 152, 'N': 107}","{'N2': 216, 'N3': 158, 'W': 145, 'N1': 132, 'R...","{'AR': 104, 'Awake': 6, 'AR,AR': 2, 'L Out': 1..."
8,5,1,2009-02-14,67.0,170.0,159.0,72.0,49.0,Female,874,93,70.0,98,186.0,96.512586,83.615561,"{'B': 483, 'R': 287, 'L': 104}","{'N2': 285, 'W': 257, 'N3': 191, 'R': 81, 'N1'...","{'AR': 40, 'PLM': 22, 'AR,LM': 10, 'LM': 10, '..."
9,5,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,903,93,70.0,98,186.0,96.516058,83.863787,"{'B': 494, 'R': 291, 'L': 118}","{'N2': 305, 'W': 268, 'N3': 143, 'R': 95, 'N1'...","{'AR': 78, 'Awake': 10, 'MChg': 2, 'AR,AR': 1,..."


In [20]:
# Normalize the JSON data into a flat table
final_df[["Stage_Summary","Sex","ID","Age"]]

# Export the normalized DataFrame to a CSV file
# df_normalized.to_csv('sleep_stage_output_subgroup_3.csv', index=False)

,Stage_Summary,Sex,ID,Age
0,"{'N2': 240, 'R': 109, 'N3': 106, 'N1': 56, 'W'...",Male,1,30.0
1,"{'N2': 257, 'N3': 103, 'R': 103, 'N1': 56, 'W'...",NaN,1,NaN
2,"{'N2': 305, 'N3': 197, 'R': 149, 'N1': 122, 'W...",Male,2,41.0
3,"{'N2': 294, 'N3': 242, 'R': 163, 'N1': 93, 'W'...",NaN,2,NaN
4,"{'N3': 317, 'N2': 248, 'R': 103, 'W': 88, 'N1'...",Male,3,52.0
5,"{'N3': 288, 'N2': 260, 'R': 118, 'W': 80, 'N1'...",NaN,3,NaN
6,"{'N2': 232, 'W': 162, 'N3': 159, 'N1': 133, 'R...",Male,4,58.0
7,"{'N2': 216, 'N3': 158, 'W': 145, 'N1': 132, 'R...",NaN,4,NaN
8,"{'N2': 285, 'W': 257, 'N3': 191, 'R': 81, 'N1'...",Female,5,49.0
9,"{'N2': 305, 'W': 268, 'N3': 143, 'R': 95, 'N1'...",NaN,5,NaN


In [40]:
import pandas as pd
from ast import literal_eval

df = pd.read_csv('summary_data.csv')

df['Stage_Summary'] = df['Stage_Summary'].apply(lambda x: literal_eval(x))

stage_summary_expanded = pd.json_normalize(df['Stage_Summary'])

# Drop the 'BPOS_Summary', 'Events_Summary', and original 'Stage_Summary' columns
df = df.drop(['BPOS_Summary', 'Events_Summary', 'Stage_Summary'], axis=1)

# Combine the expanded stage summary with the original DataFrame, now without the dropped columns
df_combined = pd.concat([df, stage_summary_expanded], axis=1)

# Assuming 'ID' and 'Sex' are not included in the mean calculation
numeric_cols = df_combined.select_dtypes(include=['float64', 'int64']).columns.difference(['ID'])
df_grouped = df_combined.groupby('ID')[numeric_cols].mean().reset_index()

# Merge non-numeric data ('Sex') back into the grouped DataFrame
df_final = pd.merge(df_grouped, df[['ID', 'Sex']].drop_duplicates('ID'), on='ID', how='left')

print(df_final.head())


   ID   Age  Average_HR  Average_SpO2  Expert  Height_cm  Height_in  Max_HR  \
0   1  64.0   63.200229     93.279976     1.5      172.0       68.0    91.0   
1   2  52.0   66.961602     95.410666     1.5      174.0       69.0   162.0   
2   3  38.0   67.842759     99.301901     1.5      172.0       68.0    99.0   
3   4  27.0   59.204995     94.752488     1.5      178.0       70.0   152.0   
4   5  58.0   53.651972     97.512307     1.5      145.0       57.0   122.0   

   Max_SpO2  Min_HR  ...     N2     N3      R  Total_Epoch    U      W  \
0      99.0    52.0  ...  201.5  228.0  118.5        871.5  NaN  266.0   
1      98.0    57.0  ...  378.5  131.0  102.0        937.5  NaN  216.5   
2     100.0    53.0  ...  308.0  159.5  239.0        922.5  2.0  112.0   
3      97.0    47.0  ...  122.5  154.0   28.0        320.5  1.0    6.0   
4     100.0    44.0  ...  288.5  164.5   43.5        873.5  NaN  302.5   

   Weight_kg  Weight_lbs   n2     Sex  
0       83.0       183.0  NaN    Male  


In [41]:
df_final.to_csv("test_2.csv")

In [36]:
stage_summary_expanded

""
0
1
2
3
4
5
6
7
8
9
